## Regression con el dataset BIWI

Este es un ejemplo más avanzado para mostrar como crear bases de datos costumizadas y hace regrecion con imagenes. La tarea es escontrar las coordenadas del centro de la cabeza en cada imagen. La data proviene del [BIWI head pose dataset](https://data.vision.ee.ethz.ch/cvl/gfanelli/head_pose/head_forest.html#db), gracias a Gabriele Fanelli et al. Aqui se convirtieron las imagen a formato jpeg, lo que hay que bajar la base de datos desde [este link](https://s3.amazonaws.com/fast-ai-imagelocal/biwi_head_pose.tgz).

In [ ]:
!mkdir ~/.fastai
!mkdir ~/.fastai/data
!mkdir ~/.fastai/data/biwi_head_pose
!ln -s /data/home/instructor1/.fastai/data/biwi_head_pose/0* ~/.fastai/data/biwi_head_pose/
!ln -s /data/home/instructor1/.fastai/data/biwi_head_pose/1* ~/.fastai/data/biwi_head_pose/
!ln -s /data/home/instructor1/.fastai/data/biwi_head_pose/2* ~/.fastai/data/biwi_head_pose/

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#import torch
#torch.cuda.set_device(2)
from fastai.vision import *

## Obteniendo y convirtiendo la data

In [ ]:
#path = untar_data(URLs.BIWI_HEAD_POSE)
path = Config.data_path()/'biwi_head_pose'

In [ ]:
cal = np.genfromtxt(path/'01'/'rgb.cal', skip_footer=6); cal

In [ ]:
fname = '09/frame_00667_rgb.jpg'

In [ ]:
def img2txt_name(f): return path/f'{str(f)[:-7]}pose.txt'

In [ ]:
img = open_image(path/fname)
img.show()

In [ ]:
ctr = np.genfromtxt(img2txt_name(fname), skip_header=3); ctr

In [ ]:
def convert_biwi(coords):
    c1 = coords[0] * cal[0][0]/coords[2] + cal[0][2]
    c2 = coords[1] * cal[1][1]/coords[2] + cal[1][2]
    return tensor([c2,c1])

def get_ctr(f):
    ctr = np.genfromtxt(img2txt_name(f), skip_header=3)
    return convert_biwi(ctr)

def get_ip(img,pts): return ImagePoints(FlowField(img.size, pts), scale=True)

In [ ]:
get_ctr(fname)

In [ ]:
ctr = get_ctr(fname)
img.show(y=get_ip(img, ctr), figsize=(6, 6))

## Creando el dataset

In [ ]:
data = (PointsItemList.from_folder(path)
        .split_by_valid_func(lambda o: o.parent.name=='13')
        .label_from_func(get_ctr)
        .transform(get_transforms(), tfm_y=True, size=(120,160))
        .databunch().normalize(imagenet_stats)
       )

In [ ]:
data.show_batch(3, figsize=(9,6))

## Entrenar el modelo

In [ ]:
learn = cnn_learner(data, models.resnet34)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr = 2e-2

In [ ]:
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1')

In [ ]:
learn.load('stage-1');

In [ ]:
learn.show_results()

## Data augmentation

In [ ]:
tfms = get_transforms(max_rotate=20, max_zoom=1.5, max_lighting=0.5, max_warp=0.4, p_affine=1., p_lighting=1.)

data = (PointsItemList.from_folder(path)
        .split_by_valid_func(lambda o: o.parent.name=='13')
        .label_from_func(get_ctr)
        .transform(tfms, tfm_y=True, size=(120,160))
        .databunch().normalize(imagenet_stats)
       )

In [ ]:
def _plot(i,j,ax):
    x,y = data.train_ds[0]
    x.show(ax, y=y)

plot_multi(_plot, 3, 3, figsize=(8,6))